In [1]:
import requests
import os
from dotenv import load_dotenv

In [2]:
os.listdir()

['NYT_sentiment.ipynb',
 'crypto_sentiment.ipynb',
 'crypto_sentiment.pdf',
 '.env',
 '.ipynb_checkpoints']

In [3]:
load_dotenv()

True

In [4]:
NYT_API=os.getenv('NYT_API')

In [5]:
topic='Tesla'
url=f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={topic}&api-key={NYT_API}'
response=requests.get(url)
data=response.json()['response']['docs'][0]

In [6]:
headlines=[]
i=1
max_results=100
topic='Tesla'
# begin_date=20120101&end_date=20121231
beg_date='20190601'
end_date='20191230'
for n in range(1, int(max_results/10)): 
    url=f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={topic}&api-key={NYT_API}&page={n}&begin_date={beg_date}&end_date={end_date}'
    response=requests.get(url)
    data=response.json()
    article_list=data['response']['docs']
    for each_article in article_list: 
        article_dict={}
        article_dict['item']=i
        article_dict['headline']=each_article['headline']['main']
        article_dict['pub_date']=each_article['pub_date']
        headlines.append(article_dict)
        i=i+1

In [7]:
import pandas as pd
df=pd.DataFrame(headlines)
df

,item,headline,pub_date
0,1,Tesla Batteries Investigated for Possible Defects,2019-11-02T01:28:00+0000
1,2,"Tesla Reports Profit for Quarter, Sending Shar...",2019-10-23T21:21:27+0000
2,3,Elon Musk Bids Twitter Farewell — Briefly,2019-11-02T02:44:01+0000
3,4,Tesla’s Record Deliveries Aren’t Enough for In...,2019-10-02T20:52:59+0000
4,5,A Police Tesla Nearly Ran Out of Power During ...,2019-10-03T09:00:09+0000
...,...,...,...
85,86,Behind a U.A.W. Crisis: Lavish Meals and Luxur...,2019-12-26T08:00:15+0000
86,87,How Young Kim Is Trying to Reclaim Orange Coun...,2019-10-04T13:01:29+0000
87,88,These High-Tech Sensors May Be the Key to Auto...,2019-09-26T10:00:05+0000
88,89,The Ups and Downs of Life Without Wheels of My...,2019-09-17T15:01:00+0000


In [8]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(headlines):
    # print(headlines)
    sentiment_by_word=[]
    sentiment_by_sentence=[]
    for each_headline in headlines:
        article_score=[]
        sentiment_by_sentence.append(sid.polarity_scores(each_headline)["compound"])
        for each_word in each_headline.split(' '):
            if each_word == None:
                continue
            else:
                # print(f'{each_word}-{sid.polarity_scores(each_word)}')
                article_score.append(sid.polarity_scores(each_word)["compound"])
                # 
        sentiment_by_word.append(sum(article_score) / len(article_score))
        # print(article_score)
        # print('-'*20)
    return sentiment_by_word, sentiment_by_sentence

In [9]:
results=headline_sentiment_summarizer_avg(df['headline'])
df['sent_by_word_score']=results[0]
df['sent_by_sentence_score']=results[1]

In [10]:
df

,item,headline,pub_date,sent_by_word_score,sent_by_sentence_score
0,1,Tesla Batteries Investigated for Possible Defects,2019-11-02T01:28:00+0000,-0.066983,-0.4019
1,2,"Tesla Reports Profit for Quarter, Sending Shar...",2019-10-23T21:21:27+0000,0.092050,0.6249
2,3,Elon Musk Bids Twitter Farewell — Briefly,2019-11-02T02:44:01+0000,0.000000,0.0000
3,4,Tesla’s Record Deliveries Aren’t Enough for In...,2019-10-02T20:52:59+0000,0.000000,0.0000
4,5,A Police Tesla Nearly Ran Out of Power During ...,2019-10-03T09:00:09+0000,-0.025119,-0.4019
...,...,...,...,...,...
85,86,Behind a U.A.W. Crisis: Lavish Meals and Luxur...,2019-12-26T08:00:15+0000,-0.069433,-0.6249
86,87,How Young Kim Is Trying to Reclaim Orange Coun...,2019-10-04T13:01:29+0000,0.000000,0.0000
87,88,These High-Tech Sensors May Be the Key to Auto...,2019-09-26T10:00:05+0000,0.000000,0.0000
88,89,The Ups and Downs of Life Without Wheels of My...,2019-09-17T15:01:00+0000,0.000000,0.0000
